匯入 multiprocessing

In [ ]:
import multiprocessing as mp
#import threading as td

建立process，用法和threading相同

In [ ]:
def job(a,d):
    print(a*d)

if __name__=='__main__':
  #t1 = td.Thread(target=job,args=(1,2))
  p1 = mp.Process(target=job,args=(1,2))
  #t1.start()
  p1.start()
  #t1.join()
  p1.join()

queue : 用法和threading相似

In [ ]:
def job(q):
    res = 0
    for i in range(1000):
        res += i+i**2+i**3
    q.put(res)  # queue


if __name__ == '__main__':
    q = mp.Queue()# queue為mp中的一個物件
    p1 = mp.Process(target=job, args=(q,))
    p2 = mp.Process(target=job, args=(q,))
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    res1 = q.get()
    res2 = q.get()
    print(res1+res2)


效率比較：threading & multiprocessing & normal

In [ ]:
import multiprocessing as mp
import threading as td
import time


def job(q):
    res = 0
    for i in range(1000000):
        res += i+i**2+i**3
    q.put(res)  # queue


def multicore():
    q = mp.Queue()
    p1 = mp.Process(target=job, args=(q,))
    p2 = mp.Process(target=job, args=(q,))
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    res1 = q.get()
    res2 = q.get()
    print('multicore:', res1+res2)


def normal():
    res = 0
    for _ in range(2):
        for i in range(1000000):
            res += i+i**2+i**3
    print('normal:', res)


def multithread():
    q = mp.Queue()
    t1 = td.Thread(target=job, args=(q,))
    t2 = td.Thread(target=job, args=(q,))
    t1.start()
    t2.start()
    t1.join()
    t2.join()
    res1 = q.get()
    res2 = q.get()
    print('multithread:', res1+res2)


if __name__ == '__main__':
    st = time.time()
    normal()
    st1 = time.time()
    print('normal time:', st1 - st)
    multithread()
    st2 = time.time()
    print('multithread time:', st2 - st1)
    multicore()
    print('multicore time:', time.time()-st2)


pool：將需要運行的進程都丟進去這個池裡面，python會自動處理這些問題

In [ ]:
def job(x):
    return x*x #使用pool時所處理的工作可以有回傳值


def multicore():
    pool = mp.Pool(processes=2)  # processes 可以指定使用核心數，預設為全部核心
    res = pool.map(job, range(10)) # map()輸入函數與輸入值，會自動分配給CPU的核心
    print(res)
    res = pool.apply_async(job, (2,)) # apply_async()一次只能輸入一個值，且只會放入一個核心運算
    print(res.get())
    multi_res = [pool.apply_async(job, (i,)) for i in range(10)] # 若想利用apply_async()輸入多參數，可以利用for迴圈達成
    print([res.get() for res in multi_res])


if __name__ == '__main__':
    multicore()


共享參數：Value()
| Type code | C Type             | Python Type       | Minimum size in bytes |
| --------- | ------------------ | ----------------- | --------------------- |
| `'b'`     | signed char        | int               | 1                     |
| `'B'`     | unsigned char      | int               | 1                     |
| `'u'`     | Py_UNICODE         | Unicode character | 2                     |
| `'h'`     | signed short       | int               | 2                     |
| `'H'`     | unsigned short     | int               | 2                     |
| `'i'`     | signed int         | int               | 2                     |
| `'I'`     | unsigned int       | int               | 2                     |
| `'l'`     | signed long        | int               | 4                     |
| `'L'`     | unsigned long      | int               | 4                     |
| `'q'`     | signed long long   | int               | 8                     |
| `'Q'`     | unsigned long long | int               | 8                     |
| `'f'`     | float              | float             | 4                     |
| `'d'`     | double             | float             | 8                     |

In [ ]:
value1 = mp.Value('i', 0)
value2 = mp.Value('d', 3.14)
array = mp.Array('i', [1, 2, 3, 4]) #不同於 python 的 list 只能為一維矩陣


Lock：解決共享資源爭奪的問題

without lock

In [ ]:
def job(v, num):
    for _ in range(10):
        time.sleep(0.1)  # 暫停0.1秒
        v.value += num  # v.value 取得共享變數
        print(v.value)


def multicore():
    v = mp.Value('i', 0)  # 定義共享變數
    p1 = mp.Process(target=job, args=(v, 1))
    p2 = mp.Process(target=job, args=(v, 3))  # 設定不同數字檢視資源分配
    p1.start()
    p2.start()
    p1.join()
    p2.join()


if __name__ == '__main__':
    multicore()


with lock

In [ ]:
def job(v, num, l):
    l.acquire()
    for _ in range(10):
        time.sleep(0.1)
        v.value += num
        print(v.value)
    l.release()


def multicore():
    l = mp.Lock()
    v = mp.Value('i', 0)
    p1 = mp.Process(target=job, args=(v, 1, l))
    p2 = mp.Process(target=job, args=(v, 3, l))
    p1.start()
    p2.start()
    p1.join()
    p2.join()


if __name__ == '__main__':
    multicore()
